In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-10"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
17374,2024-11-10,Espanha Acb,08:30,Basquet Girona,Bilbao,2.43,1.57,157.5,1.87,1.92,2.5,1.98,1.77,pADfveQs,0.411523,0.636943,0.534759,0.520833,0.048465,0.0,0.2,NaN,NaN,311.620,201.970325,0.648130,219.896,86.786775,0.394672,129.770,187.144,177.00,147.84,0.0,0.0,0.0,0.0,0.304056,0.018657,0.079196,-1.92,-0.384,-3.723958,0.582589,0.4,-0.182589,-0.91,-0.182,-3.131868,0.365439,0.2,-0.165439
17375,2024-11-10,Espanha Acb,08:30,Real Madrid,Gran Canaria,1.13,6.25,161.5,1.81,1.93,-12.5,2.00,1.75,tx73xZff,0.884956,0.160000,0.552486,0.518135,0.044956,0.4,0.0,NaN,NaN,102.420,14.137555,0.138035,133.672,44.320195,0.331559,98.500,136.018,97.18,202.76,0.0,0.0,0.0,0.0,0.981135,0.045376,0.094281,0.47,0.094,1.382979,0.907693,0.9,-0.007693,-5.00,-1.000,-5.250000,0.479261,0.3,-0.179261
17376,2024-11-10,Espanha Acb,13:00,Baskonia,Lleida,1.18,4.98,172.5,1.88,1.86,-12.5,2.01,1.68,G2yftCW8,0.847458,0.200803,0.531915,0.537634,0.048261,0.4,0.0,NaN,NaN,191.092,100.212712,0.524421,283.350,152.184656,0.537091,131.588,0.000,101.12,204.93,0.0,0.0,0.0,0.0,0.872404,0.007563,0.126474,-0.30,-0.060,-3.000000,0.734844,0.7,-0.034844,0.00,0.000,inf,0.000000,0.0,0.000000
17377,2024-11-10,Espanha Acb,13:00,Leyma Coruna,Tenerife,2.71,1.47,166.5,1.86,1.94,4.5,1.95,1.85,nZ9BzDP6,0.369004,0.680272,0.537634,0.515464,0.049276,0.0,0.8,NaN,NaN,527.502,333.411973,0.632058,149.672,48.442954,0.323661,0.000,141.258,238.70,109.81,0.0,0.0,0.0,0.0,0.419527,0.029773,0.037216,0.00,0.000,inf,0.000000,0.0,0.000000,1.23,0.246,1.910569,0.522087,0.7,0.177913
17378,2024-11-10,Espanha Acb,14:30,Breogan,Barcelona,5.15,1.17,167.5,1.80,1.94,9.5,1.99,1.76,CUU2vj1L,0.194175,0.854701,0.555556,0.515464,0.048876,0.6,0.8,NaN,NaN,194.878,60.061047,0.308198,137.206,46.690270,0.340293,160.080,151.374,208.26,215.04,0.0,0.0,0.0,0.0,0.890597,0.052938,0.086738,-3.45,-0.690,-6.014493,0.513233,0.3,-0.213233,-0.85,-0.170,-1.000000,0.667144,0.5,-0.167144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17540,2024-11-10,Polônia 1. Liga,14:00,MKKS Koszalin,Inowroclaw,2.66,1.40,165.5,1.86,1.80,4.5,2.00,1.69,6XaZkFKH,0.375940,0.714286,0.537634,0.555556,0.090226,0.4,0.0,NaN,NaN,289.816,142.653120,0.492220,188.726,55.786508,0.295595,161.798,0.000,144.20,222.70,0.0,0.0,0.0,0.0,0.438894,0.023184,0.118809,-5.00,-1.000,-1.660000,0.538237,0.2,-0.338237,0.00,0.000,inf,0.000000,0.0,0.000000
17541,2024-11-10,Rússia Superliga,10:00,Tambov,Zenit Petersburg 2,3.12,1.33,155.5,1.85,1.85,6.5,1.94,1.73,xdbMG0vl,0.320513,0.751880,0.540541,0.540541,0.072393,0.4,0.4,NaN,NaN,328.716,158.253458,0.481429,135.256,74.599633,0.551544,208.326,185.986,151.90,112.56,0.0,0.0,0.0,0.0,0.568863,0.000000,0.080922,-5.00,-1.000,-2.120000,0.505873,0.2,-0.305873,4.26,0.852,0.387324,0.000000,0.0,0.000000
17542,2024-11-10,Rússia Superliga,10:00,Temp Sumz Revda,Bars Rostov,1.07,7.50,152.5,1.85,1.85,-15.5,1.95,1.72,6eMkZ8fF,0.934579,0.133333,0.540541,0.540541,0.067913,0.2,0.2,NaN,NaN,128.076,61.190677,0.477768,177.824,84.239271,0.473723,93.260,242.686,103.95,196.48,0.0,0.0,0.0,0.0,1.061073,0.000000,0.088629,-2.87,-0.574,-0.121951,0.000000,0.0,0.000000,0.03,0.006,1083.333333,0.000000,0.0,0.000000
17543,2024-11-10,Rússia Superliga,13:00,CSKA Moscow 2,Dynamo Grozny,1.22,3.98,154.5,1.85,1.85,-13.5,1.95,1.72,bBnVEv90,0.819672,0.2512

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:30,Eua Nba,Los Angeles Clippers,Toronto Raptors,224.5,1.80,0.7297,Over
1,22:00,Eua Nba,Denver Nuggets,Dallas Mavericks,230.5,1.82,0.6982,Over
2,17:00,Eua Ncaa,UAB,SE Louisiana,153.5,1.91,0.9912,Over
3,23:00,Eua Nba,Portland Trail Blazers,Memphis Grizzlies,225.5,1.93,0.9481,Over
4,23:30,Eua Nba,Los Angeles Lakers,Toronto Raptors,234.5,1.81,0.9849,Over
